# Forward Backward and FISTA.

In this notebook we will code the Forward-Backward aLgorithm and its inertial an accelerated version FISTA to solve several optimization problems associated for example to the denoising and the inpaiting problem. I provide two images but you can use one of your own.

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
import requests
import numpy.linalg as npl

from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
im=chargeData('Canaletto')
im2=chargeData('Lenna')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Image(im).opts(**options),hv.Image(im2).opts(**options))

In [ ]:
def PSNR(I,Iref):
    temp=I.ravel()
    tempref=Iref.ravel()
    NbP=I.size
    EQM=np.sum((temp-tempref)**2)/NbP
    b=np.max(np.abs(tempref))**2
    return 10*np.log10(b/EQM)

# Inpainting

For the inpainting problem, we suppose we have data $y=Mx^0+b$ which are images of a target image $x^0$ through a Masking operator $M$ and we will consider also that data may be corrupted by an additional noise $b$. We want to estimate $x^0$ solving the following optimization problem 
\begin{equation}\label{Eq1}
\frac{1}{2}\Vert Mx-y\Vert^2+\lambda \Vert Tx\Vert_1
\end{equation}
where $T$ is an orthogonal transformation such that $Tx^0$ is sparse. If $x^0$ is piecewise smooth, we can use an orthogonal wavelet transform such as Daubechies 2 or 4. To perform the FB algorithm, we need to compute the gradient of $\frac{1}{2}\Vert Mx-y\Vert^2$ and the proximal operator $\lambda \Vert Tx\Vert_1$. Some help is provioded further in the notebook.

If the noise $b$ vanishes, i.e $y=Mx^0$ where $M$ is the masking operator, it may be better to solve the following optimization problem :
\begin{equation}\label{Eq2}
\underset{x}{\min}\Vert Tx\Vert_1\text{ under the constraint }y=Mx  
\end{equation}
This problem can't be solved directly using the FB algorithm since it can't be stated as the sum of a differentiable function which gradient is Lipschitz and of a function which proximal operator is known.

Moreover, in any pratical problem, there is actually some noise, even it can be small.

It can be shown that solutions of \eqref{Eq1} converge to solutions of \eqref{Eq2} when $\lambda$ tends to 0. 
The advantage of \eqref{Eq1} is that it can be solved using FB.
Solving \eqref{Eq1} allows to inpaint and to regularize (somehow remove some noise). The regularization effect grows with $\lambda$. In many practical case, with small noise, $\lambda$ must be chosen quiete small in \eqref{Eq1}. 
The main drawback of such a choice is that it slows down the algorithm, an higher number of iterations is needed to get a good numerical result. When we use these methods, we must be aware that the choice of $\lambda$ of the number of iterations and the starting point of this algorithm is essential. If the numerical resultats are not as nice as you wish, you may have to change the value of $\lambda$ or increase the number of iterations.



# A few known results on explicit Gradient Descent and Forward-backward (FB)

If $F=f+g$ is a convex fucntion, sum of two convex functions, $f$ a differentiable function which gradient is $L-$Lipschitz and $g$ a convex function such that $prox_g$ is known, the Forward-Backward algorithm is defined by 
$$x_{n+1}=prox_{hg}(x_n-h\nabla f(x_n))=Tx_n\quad \text{ with }T:=prox_{hg}\circ (Id-h\nabla f)$$
One can show that the sequence $(F(x_n)-F(x^*))_{n\in\mathbb{N}}$ is non increasing and moreover 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn}$$
This decay rate $\frac{1}{n}$ is optimal in the sens that it is impossible to get a bound decaying like $\frac{1}{n^{\delta}}$ with $\delta>1$ for all convex fucntions. Nevertheless it can be proved that if $h<\frac{1}{L}$ one actually has
$$F(x_n)-F(x^*)=o\left(\frac{1}{n}\right).$$  
If the function $f$ is strongly convex the decay is geometrical.

# Some rermarks on iteratives algorithms and FB in particular.

When you use an iterative algorithm depending on one or more parameters, you must be careful to evaluate the results, the outputs of the algorithm. It is crucial to check some points :

1) Check that the algorithm is close to converge... that is check that enough iterations have been done. This number of iterations may change depending on parameters and on the starting point $x_0$ chosen to build ythe sequence.

To check this point, I Invite you to display the curve of the values $F(x_n)$ and to think twice about what you doing and what you see on the outputs. For example, if you are performing inpainting and if some black pixels left, or if the sequence $F(x_n)$ is still decaying quite fast, it means that it may be relevant to increse the number of iterations. More generally, observing the artefacts of defects on the output may give some clues of what can be done to improve the result. Moreover, have a look to the sequence $F(x_n)$ may indicate that less iterations could have been performed to get a similar output.

2) Explore the possible values of paramters. It may occur that some values of parameters provide some relevant results and some don't. Before pretending that an algorithm is not efficient you must ensure that the parameters have been chosen wisely. These parameters can be the ones of the fucntion to minimize such as $\lambda$ in \eqref{Eq1} or the internal parameters of the algorithm such as the descent step for example. Making an Experiments Plan may be a good way to estimate the values of good parameters.

One can observe that the value of $F(x)$ is not the only criterion to evaluate the output of an algorithm, actually for different values of $\lambda$, the PSNR may be more relevant. We can't avoid thinking... 

3) When it it is possible choose a good starting point. Iterative algorithms build images (or signals) which converges to a minimiozer of the fucntion $F$. the choice of the staring point may be crucial. 
If we do not have any information, we can choose the constant image 0 or any relevant image, for example the masked image if we are performing inapainting or the noisy image if we are performing denoising. If for any problem a simple and quick method may provide a rough or innacurate output, it can be used as a starting point of the method. For the inpainting, we may start using a simple median filtering. The image is divided into small squares and the holes are filled up with the median value of the observed data.

One can often distinguish two steps during the optimization of the function $F$, a first one to go from the starting point to a good image, a coarse approximation of the whished output and a second one from the coarse approximation to the whished output. It may be difficult to avoid the second step but the first one may be shortened using a good starting point.

In many situations to compare several methods with various parameters or wavelet bases, it can be useful to compute the PSNR of each output on a benchmark to get a quite fair criterion for the comparison.


## Forward Backward for the inpainting.

We can start using observations without any noise and apply a mask. To create a 2D mask one can use the following commands : 

In [ ]:
np.random.seed(seed=1)
n1,n2=np.shape(im)
r=np.random.rand(n1,n2)
M=(r<0.5)
M=M*1.0

We can of course change the proportion of pixels that are masked.

We can mask the image by the following command :


In [ ]:
temp=M*im
pn.Row(hv.Image(im).opts(cmap='gray',width=400,height=400,title="Image originale"),
       hv.Image(temp).opts(cmap='gray',width=400,height=400,title="Image masquée"))

### Initilisation by median filter.

Implement and test a median filter which inpaint roughly the image replacing missing pixels using the median of values of observed pixels on vois*vois squares.

In [ ]:
def FiltreMedian(im,vois) : 
    imrec=np.copy(im)
    n1,n2=np.shape(im)
    K1=int(np.floor(n1/vois))
    K2=int(np.floor(n2/vois))  
    for k1 in range (K1) : 
        for k2 in range (K2) : 
            voisinage = np.copy(im[k1*vois:(k1+1)*vois, k2*vois:(k2+1)*vois])
            med_vois = np.median(voisinage[voisinage > 0])
            for v1 in range(vois) : 
                for v2 in range (vois) : 
                    if voisinage[v1,v2]== 0 : 
                        voisinage[v1,v2]=med_vois
            imrec[k1*vois:(k1+1)*vois, k2*vois:(k2+1)*vois] = voisinage
    return(imrec) 

In [ ]:
imfiltre = FiltreMedian(temp, 16)
pn.Row(hv.Image(temp).opts(cmap='gray', xaxis=None, yaxis=None, width=350, height=350, title="Image masquée"), 
       hv.Image(imfiltre).opts(cmap='gray',xaxis=None,yaxis=None, width=350, height=350, title="Image FiltreMedian"))

### Back to FB

Implement a function computing the gradient of the function $f(x)=\frac{1}{2}\Vert Mx-y\Vert^2$ :

M est un projeté orthogonal donc $M^\top=M=M^2$

$$
\begin{aligned}
\nabla f(x) &= M^\top (M x - y) \\
            &= M^\top Mx - M^\top y \\
            &= Mx - My \\
            &= M(x-y)
\end{aligned}
$$

In [ ]:
def GradientInpainting(x,b,M):
    g=M*(x-b)
    return g

For which value of $L$ this gradient is $L-$Lipschitz ?

* On projette sur l'espace vectoriel des pixels non nuls donc M est un projecteur orthogonal.
* De plus la norme d'opérateur est égale à 1.
* Donc L=1

The following functions compute the proximal operator of the function $g(x)=\lambda \Vert Tx\Vert_1$ where $T$ is an orthogonal wavelet transform.

In [ ]:
def SeuillageDouxOndelettes(I,wave,Seuil):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    temp=pywt.threshold(arr,Seuil,mode='soft')
    test=pywt.unravel_coeffs(temp, coeff_slices, coeff_shapes, output_format='wavedecn')
    Irec=pywt.waverecn(test, wave,mode='per')
    return Irec

def Normel1Ondelettes(I,wave):
    L=pywt.dwt_max_level(len(I),pywt.Wavelet(wave).dec_len)
    wavelet_coeffs= pywt.wavedecn(I, wave, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(wavelet_coeffs)
    norml1=sum(np.abs(arr))
    return norml1

In [ ]:
wave='haar'
imrec=SeuillageDouxOndelettes(im,wave,3)

Implement and test an inpainting program using the FB algorithm minimizing the function \eqref{Eq1}.
Don't forget to clip the data at the end of the program.

In [ ]:
def ForwardBackwardInpaintingv2(y,M,step,lam,Niter,wave):
    seuil=lam*step
    xn=FiltreMedian(y,8)
    F=np.zeros(Niter)
    for i in range(Niter):
        grad = GradientInpainting(xn,y,M)
        xn_new  = xn - step * grad
        xn  = SeuillageDouxOndelettes(xn_new, wave, seuil)
        F[i] = (1/2)* npl.norm(M * xn - y)**2 + lam * Normel1Ondelettes(xn_new,wave)
    return np.clip(xn_new, 0, 255), F 

In [ ]:
Im_rec, F = ForwardBackwardInpaintingv2(temp, M, 0.1, 0.1, 100, 'haar')
hv.Curve(F)

### Commentaire :

Notre but étant de minimiser F, mission accomplie.

Implement a dashboard allowing a numerical exploration of the previous function. We could observe that small small values of $\lambda$ produce better output. Theoreticaly, if the data are noiseless, it would be better to take $\lambda$ as small as possible. You can see it if you display the PSNR of the output when you compute a number of iterations that is large enough.

The drawback of such a choice is that it slows down extremely the algorithm. If data are noisy the optimal value of $\lambda$ is proportional to the standart deviation of the noise.

Hence, you may prefer a small value of $\lambda$ but nottoo small to ensure the convergence of the algorithm.


In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']

In [ ]:
class FBInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(10,bounds=(0,300))
    lam = param.Number(10,bounds=(1,30))
    step = param.Number(0.9,bounds=(0.5,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    def view(self):
        Im = imagesRef[self.image]
        Affich_ref = hv.Image(Im).opts(cmap='gray',width=400,height=400,title="Image originale")
        
        n1, n2 = np.shape(Im)
        r = np.random.rand(n1, n2)
        M = (r > self.masquage) * 1.0
        Im_masq = Im * M
        Affich_masq = hv.Image(Im_masq).opts(cmap='gray',width=400,height=400,title="Image masquée")
        psnr_masq = PSNR(Im_masq,Im)
        
        Im_rec, F = ForwardBackwardInpaintingv2(Im_masq, M, self.step, self.lam, self.Niter, self.wave)
        Affich_rec = hv.Image(Im_rec).opts(cmap='gray',width=400,height=400,title="Image reconstruite")
        psnr_rec = PSNR(Im_rec,Im)
        
        strp1 = "%2.2f" % psnr_masq
        te1='PSNR image masquée = '
        TN1=hv.Text(0,0.2,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        strp2 = "%2.2f" % psnr_rec
        te2='PSNR image reconstruite = '
        TN2=hv.Text(0,0,te2+strp2).opts(xaxis=None,yaxis=None)
        
        return pn.Row(pn.Column(Affich_ref,Affich_masq),pn.Column(Affich_rec,TN1*TN2))   

In [ ]:
fbinpaint=FBInpaint()
pn.Row(fbinpaint.param,fbinpaint.view)

### Commentaires :

* Si on prend un lambda petit alors on résout le bon problème d'optimisation et on obtient un très bon PSNR.
* En revanche, il ne faut pas qu'il soit trop petit sinon on aura besoin d'un trop grand nombre d'itérations pour converger.
* Si le nombre d'itération est trop faible, l'algorithme n'aura pas le temps de converger et les résultats obtenus ne seront pas pertinents.
* Il faut donc ajuster lambda de telle manière à trouver le bon équilibre entre précision et rapidité : lambda potentiellement petit, mais pas trop, avec un nombre d'itérations raisonnable.

* Le pas mesure la vitesse à laquelle on va "descendre" vers la solution.
* Si le pas est trop grand, il y a un risque de dépasser le minimum et de diverger ou alors d'osciller autour du minimum.
* Si le pas est trop petit, alors l'algo aura besoin de trop d'itérations pour converger.
* Il faut donc trouver un pas ni trop grand ni trop petit.

# Accélération de Nesterov, FISTA.

Yurii Nesterov has proposed in the 80's several methods to accelerate the explicit Gradient Descent. We will focus on one of these, the one that is described in the lesson. If you have a look to research paper be aware that the 2 words "Nesterov accelerations" may have several meanings... using interpolation or extrapolation, be specific to strongly convex functions or not. it may apply to the explicit Gradient Descent (GD) or to the Forward Backward algorithm. Moreover we will not use the original parameter of Nesterov or FISTA but the ones we proposed in 2014 with Antonin Chambolle to ensure the convergence of iterates and speed up the convergence. 

the acceleration of the GD proposed but Nesterov in 19!4 and adapted to FB under the name of FISTA by Beck and Teboulle in 2009 is easy the apply and without any additional computational cost.

The idea is to apply the FB (or the GD for a single differntiable function) to a shifted point of $x_n$ with a step $h<\frac{1}{L}$. 
Be careful this bound is more restrictive than the one for the classical GD :
$$x_{n+1}=T(x_n+\alpha_n(x_n-x_{n-1}))$$ 
where the sequence $\alpha_n$ is chosen in a suitable way and $T$ is the FB operator. We say that FISTA is an inertial method because it uses an inertial term as a memory of the last descent direction.

Let's give some key points of this method :
1) The original choice of Nesterov for the sequence $\alpha_n$ is the following :
\begin{equation}
\alpha_n=\frac{t_n-1}{t_{n+1}}\text{ avec }t_1=1\text{ et }t_{n+1}=\frac{1+\sqrt{1+t_n^2}}{2}
\end{equation}
2) For this choice we have 
$$F(x_n)-F(x^*)\leqslant \frac{2\Vert x_0-x^*\Vert^2}{hn^2}$$
3) We can take the more simple choice $\alpha_n=\frac{n-1}{n+a-1}$ with $a>3$ (and that's what you will implement) and in this cas we have 
$$F(x_n)-F(x^*)\leqslant \frac{(a-1)^2\Vert x_0-x^*\Vert^2}{2h(n+a)^2}$$ and moreover 
$F(x_n)-F(x^*)=o\left(\frac{1}{n^2}\right)$ and the sequence $(x_n)_{n\geqslant 1}$ converges. 
It can be noticed, there are no inertia in the first step ($\alpha_1=0$) and thus $x_1=T(x_0)$. 
The inertia appears for the computation of $x_2$. The original choice of Nesterov is very close to the choice $\alpha=3$.

4) In the case of a composite fucntion ($F$ is a sum of a diffrentiable fucntion $f$ and a possibly non smooth fucntion $g$), this inertial algorithm is called FISTA (Fast Iterative Soft Shrinkage Thresholding Algorithm) because when $g$ is the $\ell_1$ norm, the proximal operator is a soft thresholding but the name FISTA is the name of this algorithm even if $g$ is not an $\ell_1$ norm.

5) Unlike FB, the sequence $(F(x_n)-F(x^*))_{n\geqslant 1}$ given by FISTA is not necessarily non increasing. The previous bounds on FB or FISTA are only ... bounds... but does not describe the real decay rate.
In practical cases you should see that FISTA is often faster than FB.


Implement an inpaiting algorithm minimizing the fucntion \eqref{Eq1} using FISTA.

In [ ]:
def FISTAInpainting(y,M,step,lam,Niter,wave,alpha0):
    seuil=lam*step
    xn=FiltreMedian(y,8)
    F=np.zeros(Niter)
    xn_old=xn
    for k in np.arange(0,Niter):
        alpha = k/(k+alpha0)
        xn_new = xn + alpha*(xn - xn_old)
        grad = GradientInpainting(xn_new,y,M)
        xn_old = xn
        xn = xn_new - step * grad 
        xn = SeuillageDouxOndelettes(xn, wave, seuil)
        F[k] = (1/2)* npl.norm(M * xn - y)**2 + lam * Normel1Ondelettes(xn,wave)
    return np.clip(xn,0,255),F

In [ ]:
class FISTAInpaint(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    Niter = param.Integer(10,bounds=(0,500))
    lam = param.Number(10,bounds=(0.01,10))
    step = param.Number(0.9,bounds=(0.5,4))
    masquage = param.Number(0.5,bounds=(0.1,1))
    alpha = param.Number(3,bounds=(1,10))
    def view(self):
        Im = imagesRef[self.image]
        Affich_ref = hv.Image(Im).opts(cmap='gray',width=400,height=400,title="Image originale")
        
        n1, n2 = np.shape(Im)
        r = np.random.rand(n1, n2)
        M = (r > self.masquage) * 1.0
        Im_masq = Im * M
        Affich_masq = hv.Image(Im_masq).opts(cmap='gray',width=400,height=400,title="Image masquée")
        psnr_masq = PSNR(Im_masq,Im)
        
        Im_rec, f = FISTAInpainting(Im_masq, M, self.step, self.lam, self.Niter, self.wave, self.alpha)
        Affich_rec = hv.Image(Im_rec).opts(cmap='gray',width=400,height=400,title="Image reconstruite")
        psnr_rec = PSNR(Im_rec,Im)
        
        strp1 = "%2.2f" % psnr_masq
        te1='PSNR image masquée = '
        TN1=hv.Text(0,0.2,te1+strp1).opts(xaxis=None,yaxis=None,toolbar=None)
        strp2 = "%2.2f" % psnr_rec
        te2='PSNR image reconstruite = '
        TN2=hv.Text(0,0,te2+strp2).opts(xaxis=None,yaxis=None)
        
        return pn.Row(pn.Column(Affich_ref,Affich_masq),pn.Column(Affich_rec,TN1*TN2))  

In [ ]:
fistainpaint=FISTAInpaint()
pn.Row(fistainpaint.param,fistainpaint.view)

### Commentaires :
L'algorithme de Nesterov est un cas particulier de l'accélération de Nesterov avec $\alpha$=3. Nous remarquons que les résultats sont meilleurs pour $\alpha$>3

# A retenir

If you implement a FB algorithm, nesterov accelerations are often efficient and it is always useful to test and even to test several choices of $a$. beginning with $a=3$. One can notive that, at least theoretically, the sequence   
generated $(F(x_n)-F(x^*))_{n\geqslant 1}$ generated by FB as a geometrical decay when $F$ is strongly convex, which is not the case for the studied Nesterov scheme. But it exists some accelerations that are dedicated to this specific situation. In practical cases, for a reasonnable number of iterations, these Nesterov accelerations are really competitive.